In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import collections
import matplotlib.pyplot as plt
import torch
import time
import collections
from functools import reduce

os.environ["CUDA_VISIBLE_DEVICES"]="2"#,1,2,4"

sys.path.insert(0, os.path.join("..", "..", ".."))

from petroflow import CoreBatch, CoreIndex
from petroflow.batchflow.models.torch import ResNet18
from petroflow.batchflow import Dataset, Pipeline, V, B, W, F, C
from petroflow.batchflow.research import Research, Option, KV

import utils

In [2]:
PATH = '/notebooks/data/august_dataset/cropped_wells'

## Pipelines

In [3]:
BATCH_SIZE = 8
N_EPOCH = 50
SHAPE = (3, 500, 250)

model_config = {'initial_block/inputs': 'images',
                'inputs/images/shape': SHAPE,
                'inputs/labels/classes': F(lambda x: len(x), V('labels_mapping')),
                'initial_block/inputs': 'images',
                'optimizer': 'Adam',
                'output': 'proba',
                'device': 'gpu:0',
                'loss': 'ce'#, dict(weight=V('weight')))}
               }

train_ppl = (Pipeline()
    .add_namespace(utils)
    .set_dataset(C('dataset'))
    .load(uv=False, dst=['dl'])
    .normalize(src='dl', dst='dl')
    .add_namespace(np)
    .create_labels(C('annotation').LITHOLOGY.loc)
    .update(B('labels'), B('labels').tolist())
    .to_array(src='dl', dst='images', dtype='float32')
    .make_random_crops(src=['images'], dst=['images'], channels='last', shape=SHAPE[1:], n_crops=1)
    .concatenate(B('images'), axis=0, save_to=B('images'))
    .init_variable('loss', default=None)
    .transpose(B('images'), axes=(0, 3, 1, 2), save_to=B('images'))
    .encode(B('labels'), C('labels_mapping'), save_to=B('labels'))
    .init_model('dynamic', ResNet18, 'model', model_config)
    .train_model('model', B('images'), B('labels'), use_lock=True, fetches='loss',
             save_to=V('loss', mode='w'))
    .run_later(64, n_epochs=100, shuffle=42)
)

def init_research(iteration, experiment, ppl_train, ppl_test):
    ppl = experiment[ppl_train].pipeline
    cfg = experiment[ppl_train].config.config()
    ann, ds_train, ds_test, _ = utils.get_input_data(PATH, cfg['lithology'])
    labels_mapping, reverse_mapping, counter, weights = utils.get_statistics(ds_train, ann)
    

    new_cfg = {
        'lithology': cfg['lithology'],
        'annotation': ann,
        'dataset': ds_train,
        'dataset_test': ds_test,
        'labels_mapping': labels_mapping
    }
    
    experiment[ppl_train].pipeline.set_config(new_cfg)
    experiment[ppl_test].pipeline.set_config(new_cfg)
    
    return {'annotation': ann,
            'dataset': ds_train,
            'dataset_test': ds_test,
            'labels_mapping': labels_mapping,
            'reverse_mapping': reverse_mapping,
            'counter': counter,
            'weights': weights}


def get_model(iteration, experiment, pipeline):
    return experiment[pipeline].pipeline.get_model_by_name('model')

In [4]:
test_ppl = (Pipeline()
    .add_namespace(utils)
    .set_dataset(C('dataset_test'))
    .load(uv=False, dst=['dl'])
    .normalize(src='dl', dst='dl')
    .add_namespace(np)
    .create_labels(C('annotation').LITHOLOGY.loc)
    .update(B('labels'), B('labels').tolist())
    .to_array(src='dl', dst='images', dtype='float32')
    .make_random_crops(src=['images'], dst=['images'], channels='last', shape=SHAPE[1:], n_crops=1)
    .concatenate(B('images'), axis=0, save_to=B('images'))
    .init_variable('metrics', default=None)
    .transpose(B('images'), axes=(0, 3, 1, 2), save_to=B('images'))
    .encode(B('labels'), C('labels_mapping'), save_to=B('labels'))
    .import_model('model', C('import_from'))
    .predict_model('model', B('images'), fetches='proba', save_to=B('proba'))
    .gather_metrics('class', targets=B('labels'), predictions=B('proba'),
                    fmt='proba', axis=-1, save_to=V('metrics', mode='u'))
    .run_later(64, shuffle=False, n_epochs=1, bar=False, drop_last=False)
)

# Research

In [5]:
! rm -r research

In [9]:
options = [
    ['Песчаник', 'Аргиллит'],
    ['Песчаник', 'Алевролит'],
    ['Аргиллит', 'Алевролит'],
    ['Аргиллит', 'Алевролит', 'Песчаник']
]

options = Option('lithology', [KV(item, '_'.join(item)) for item in options])

research = (Research()
            .add_grid(options)
            .add_function(init_research, execute='#0', ppl_train='train', ppl_test='test',
                          returns='mapping', name='init', dump='#0', logging=T)
            .add_pipeline(train_ppl, variables='loss', name='train')
            .add_pipeline(test_ppl, name='test', run=False, execute='last',
                          import_from='train', variables='metrics', logging=True)
            .get_metrics('test',
                         metrics_var='metrics', 
                         metrics_name='f1_score',
                         returns='f1_score',
                         execute='last')
            .add_function(get_model, execute='last', pipeline='train', returns='model')
           )

research.run(1, 5, bar=True)

Research research is starting...


  0%|          | 0/20 [00:00<?, ?it/s]

Distributor has 4 jobs with 5 iterations. Totally: 20


 25%|██▌       | 5/20 [01:29<04:29, 17.94s/it]../../../petroflow/batchflow/batchflow/models/metrics/classify.py:303: RuntimeWarning:

invalid value encountered in long_scalars

 50%|█████     | 10/20 [02:55<02:55, 17.55s/it]../../../petroflow/batchflow/batchflow/models/metrics/classify.py:303: RuntimeWarning:

invalid value encountered in long_scalars

 75%|███████▌  | 15/20 [04:14<01:24, 16.98s/it]../../../petroflow/batchflow/batchflow/models/metrics/classify.py:303: RuntimeWarning:

invalid value encountered in long_scalars

100%|██████████| 20/20 [06:12<00:00, 18.65s/it]../../../petroflow/batchflow/batchflow/models/metrics/classify.py:303: RuntimeWarning:

invalid value encountered in long_scalars

100%|██████████| 20/20 [06:13<00:00, 18.66s/it]


In [10]:
research.load_results()

,accuracy,f1_score,iteration,lithology,loss,mapping,metrics,name,repetition,sensitivity,specificity
0,NaN,NaN,0,Песчаник_Аргиллит,NaN,"{'reverse_mapping': {0: 'Песчаник', 1: 'Аргилл...",NaN,init,0,NaN,NaN
0,NaN,NaN,0,Песчаник_Аргиллит,4.487909,NaN,NaN,train,0,NaN,NaN
1,NaN,NaN,1,Песчаник_Аргиллит,2.320973,NaN,NaN,train,0,NaN,NaN
2,NaN,NaN,2,Песчаник_Аргиллит,1.889629,NaN,NaN,train,0,NaN,NaN
3,NaN,NaN,3,Песчаник_Аргиллит,1.420853,NaN,NaN,train,0,NaN,NaN
4,NaN,NaN,4,Песчаник_Аргиллит,0.832675,NaN,NaN,train,0,NaN,NaN
0,NaN,NaN,4,Песчаник_Аргиллит,NaN,NaN,<petroflow.batchflow.batchflow.models.metrics....,test,0,NaN,NaN
0,f1_score,accuracy,4,Песчаник_Аргиллит,NaN,NaN,NaN,test_metrics,0,specificity,sensitivity
0,NaN,NaN,0,Песчаник_Алевролит,NaN,"{'reverse_mapping': {0: 'Песчаник', 1: 'Алевро...",NaN,init,0,NaN,NaN
0,NaN,NaN,0,Песчаник_Алевролит,4.496798,NaN,NaN,train,0,NaN,NaN
